In [52]:
# Imports
%matplotlib inline
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

## PreProcessing Data

In [21]:
# file path - this for linux windows you will need "//"
f_path = "cardio_train.csv"

df = pd.read_csv(f_path,sep = ';',index_col=0)

In [37]:
df2 = df[['age','gender','height','weight','smoke']]
df2['gender'] = df2['gender'].replace({1:0,2:1})

C:\Users\matan\AppData\Local\Temp/ipykernel_19736/353712324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['gender'] = df2['gender'].replace({1:0,2:1})


In [38]:
'''
normalization of the data
'''
df2['age'] = df['age'].apply(lambda x: x/360)

C:\Users\matan\AppData\Local\Temp/ipykernel_19736/3830185172.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['age'] = df['age'].apply(lambda x: x/360)


In [39]:
'''
0 - female
1 - male
'''
df2,df2['gender'].value_counts()

(             age  gender  height  weight  smoke
 id                                             
 0      51.091667       1     168    62.0      0
 1      56.188889       0     156    85.0      0
 2      52.380556       0     165    64.0      0
 3      48.952778       1     169    82.0      0
 4      48.538889       0     156    56.0      0
 ...          ...     ...     ...     ...    ...
 99993  53.444444       1     168    76.0      1
 99995  62.780556       0     158   126.0      0
 99996  52.961111       1     183   105.0      0
 99998  62.308333       0     163    72.0      0
 99999  57.055556       0     170    72.0      0
 
 [70000 rows x 5 columns],
 0    45530
 1    24470
 Name: gender, dtype: int64)

In [40]:
'''
split the dataset into training_set 50K
and testing_set 20K
'''
training_set = df2.iloc[-20000:]
testing_set = df2.iloc[50000:]

In [53]:
features = 4 
alpha = 0.0001
hidden_layer_node = 4
eps = 1e-12

X = tf.placeholder(tf.float32,[None,features])
Y = tf.placeholder(tf.float32,[None,1])

#matrix, dimensions of W  
W1 = tf.Variable(tf.truncated_normal([features,hidden_layer_node],stddev = 0.1))
#vector
b1 = tf.Variable(tf.constant(0.1,shape=[hidden_layer_node]))
#
Z1 = tf.add(tf.matmul(X,W1),b1)
#ReLu activation layer
a1 = tf.nn.relu(Z1)
# 50 examples with 2 features --> 50 examples with 10 features
W2 = tf.Variable(tf.truncated_normal([hidden_layer_node,1],stddev = 0.1))
#
b2 = tf.Variable(0.)
#
Z2 = tf.matmul(a1,W2) + b2
#
sigmoid = 1 / (1.0 + tf.exp(-Z2))
#
loss = tf.reduce_mean(-(Y* tf.log(sigmoid) + (1-Y)*tf.log(1-Y+eps)))
#
update = tf.train.GradientDescentOptimizer(alpha).minimize(loss)

In [54]:
'''
transform training data into numpy array so it meets the wanted dimensions
'''
data_x = training_set.drop(columns='gender').to_numpy() #matrix of vectors, each vectors has 4 attributes
data_y = training_set.drop(columns=['age', 'weight','height','smoke']).to_numpy() #matrix of vectors, each vector has 1 attribute

In [56]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(0,100000):
    sess.run(update,feed_dict={X:data_x, Y:data_y})
    if i%10000 == 0:
        print('Iteration: ',i)
# eval evaluates the value of a variable. Therefore, K. eval(loss) gives you the value of the crossentropy loss
        print('W:\n ',sess.run(W1), '\nb:\n ', sess.run(b1), '\nLoss:\n ',loss.eval(session=sess, feed_dict={X:data_x,Y:data_y}))

Iteration:  0
W:
  [[ 0.06049263  0.08558561  0.02366725  0.00725759]
 [-0.05305875 -0.14070891 -0.0418909   0.0650982 ]
 [ 0.09768969 -0.05062525  0.19794568 -0.11103188]
 [ 0.00987921  0.00589793  0.02066181 -0.00609552]] 
b:
  [0.10000208 0.1        0.09999826 0.09999838] 
Loss:
  0.3984374
Iteration:  10000
W:
  [[ 0.1064377   0.08558561  0.009089   -0.00047174]
 [ 0.0925435  -0.14070891 -0.08801436  0.04076371]
 [ 0.16233484 -0.05062525  0.17632765 -0.12074817]
 [ 0.01006293  0.00589793  0.02060458 -0.00612555]] 
b:
  [0.1008634  0.1        0.09973415 0.09985982] 
Loss:
  0.00049886096


KeyboardInterrupt: 

In [57]:
data_x,data_y

(array([[ 52.525     , 176.        ,  72.        ,   0.        ],
        [ 64.66111111, 145.        ,  68.        ,   0.        ],
        [ 58.8       , 159.        ,  66.        ,   0.        ],
        ...,
        [ 52.96111111, 183.        , 105.        ,   0.        ],
        [ 62.30833333, 163.        ,  72.        ,   0.        ],
        [ 57.05555556, 170.        ,  72.        ,   0.        ]]),
 array([[1],
        [0],
        [1],
        ...,
        [1],
        [0],
        [0]], dtype=int64))

In [60]:
'''
preparing for testing, checking results using confusion matrix
text_x - 20k examples 
test_y - 20k labels
'''
test_x = testing_set.drop(columns='gender').to_numpy()
test_y = testing_set.drop(columns=['age', 'weight','height','smoke']).to_numpy()

#logistic regression function for prediction
# z is a vector containing 4 features [age, height, weight, smoke]
y= 1 / (1.0+tf.exp(-Z2))

# predicted will be a vector of the prediction of the testing_set
predicted = []

#looping each vector in text_x matrix
for vec in test_x:
    pr = sigmoid.eval(session=sess,feed_dict={X:[vec]})
    print(pr)
    predicted.append(1) if pr >= 0.5 else predicted.append(0)       

[[0.9989924]]
[[0.99839646]]
[[0.9985152]]
[[0.9991571]]
[[0.9992235]]
[[0.99860865]]
[[0.9995171]]
[[0.997139]]
[[0.99945563]]
[[0.9978376]]
[[0.9983419]]
[[0.9982759]]
[[0.99912983]]
[[0.99864095]]
[[0.9994]]
[[0.9988336]]
[[0.99905986]]
[[0.99731714]]
[[0.9985662]]
[[0.9974638]]
[[0.9983706]]
[[0.99820995]]
[[0.99919873]]
[[0.9994609]]
[[0.99937624]]
[[0.9979831]]
[[0.9992021]]
[[0.99890494]]
[[0.99946755]]
[[0.998615]]
[[0.9987662]]
[[0.99889296]]
[[0.9984068]]
[[0.9968105]]
[[0.9991208]]
[[0.9979062]]
[[0.99875116]]
[[0.9991841]]
[[0.99727446]]
[[0.9985097]]
[[0.9989912]]
[[0.9974331]]
[[0.99903846]]
[[0.9997677]]
[[0.99875844]]
[[0.9984256]]
[[0.9983851]]
[[0.99802893]]
[[0.99903667]]
[[0.99819547]]
[[0.9971903]]
[[0.9986803]]
[[0.9982817]]
[[0.9982608]]
[[0.9988293]]
[[0.99850583]]
[[0.9983581]]
[[0.99918455]]
[[0.99800974]]
[[0.9984585]]
[[0.99894017]]
[[0.99840087]]
[[0.9985728]]
[[0.99892765]]
[[0.9989466]]
[[0.9982822]]
[[0.99890554]]
[[0.99901974]]
[[0.99931884]]
[[0.998851

[[0.9989742]]
[[0.9982628]]
[[0.9982316]]
[[0.9972481]]
[[0.9991992]]
[[0.9994216]]
[[0.9997571]]
[[0.9987753]]
[[0.99872977]]
[[0.99752396]]
[[0.9978593]]
[[0.99938715]]
[[0.99894804]]
[[0.9985991]]
[[0.99904305]]
[[0.998204]]
[[0.9977324]]
[[0.9986021]]
[[0.99959177]]
[[0.9987544]]
[[0.9982963]]
[[0.9982591]]
[[0.99889475]]
[[0.9984372]]
[[0.9987674]]
[[0.99876463]]
[[0.9987425]]
[[0.9990125]]
[[0.9992397]]
[[0.9992797]]
[[0.999246]]
[[0.99845076]]
[[0.9974371]]
[[0.9982552]]
[[0.99790704]]
[[0.9985783]]
[[0.9988261]]
[[0.9959138]]
[[0.9992867]]
[[0.9988808]]
[[0.9985165]]
[[0.99832183]]
[[0.9986432]]
[[0.9986885]]
[[0.9991258]]
[[0.99826044]]
[[0.9992003]]
[[0.9994429]]
[[0.99948955]]
[[0.9992656]]
[[0.998855]]
[[0.9991541]]
[[0.99914193]]
[[0.9993761]]
[[0.9988482]]
[[0.9987915]]
[[0.9989517]]
[[0.9989717]]
[[0.9995043]]
[[0.9990539]]
[[0.9991806]]
[[0.99936813]]
[[0.99863005]]
[[0.99806327]]
[[0.99892324]]
[[0.9988977]]
[[0.9991916]]
[[0.9989035]]
[[0.99921834]]
[[0.9995881]]
[[0.

[[0.998243]]
[[0.99901974]]
[[0.99887675]]
[[0.99825186]]
[[0.99897325]]
[[0.9986524]]
[[0.99900717]]
[[0.9996747]]
[[0.9972451]]
[[0.99801815]]
[[0.99924517]]
[[0.99863297]]
[[0.99791807]]
[[0.9994116]]
[[0.99914384]]
[[0.9972996]]
[[0.9996221]]
[[0.99824405]]
[[0.9979219]]
[[0.9997981]]
[[0.9977689]]
[[0.9962076]]
[[0.99788874]]
[[0.99940324]]
[[0.9991315]]
[[0.99873143]]
[[0.99941397]]
[[0.99942255]]
[[0.99873656]]
[[0.99919397]]
[[0.9990872]]
[[0.9988875]]
[[0.99877673]]
[[0.9984989]]
[[0.99816966]]
[[0.99919826]]
[[0.9982418]]
[[0.99928015]]
[[0.9988939]]
[[0.99941504]]
[[0.9954032]]
[[0.9998599]]
[[0.99932396]]
[[0.9995522]]
[[0.9991186]]
[[0.9994586]]
[[0.9988426]]
[[0.9996475]]
[[0.99915516]]
[[0.99791044]]
[[0.99900657]]
[[0.9983563]]
[[0.9989845]]
[[0.99974495]]
[[0.9992299]]
[[0.99943787]]
[[0.998468]]
[[0.99868435]]
[[0.9994148]]
[[0.9990089]]
[[0.99780494]]
[[0.9991358]]
[[0.9995863]]
[[0.99896276]]
[[0.998998]]
[[0.99910504]]
[[0.99913603]]
[[0.9995586]]
[[0.9986687]]
[[0

[[0.99836546]]
[[0.9984195]]
[[0.9992274]]
[[0.99854904]]
[[0.9991007]]
[[0.997401]]
[[0.99797744]]
[[0.9978613]]
[[0.9989071]]
[[0.9977982]]
[[0.9985008]]
[[0.99861205]]
[[0.9985935]]
[[0.9989311]]
[[0.99927765]]
[[0.99929976]]
[[0.9964143]]
[[0.99897325]]
[[0.9993765]]
[[0.9987527]]
[[0.99928904]]
[[0.99929106]]
[[0.9990909]]
[[0.99886155]]
[[0.9986817]]
[[0.9981548]]
[[0.9992805]]
[[0.9987502]]
[[0.99929595]]
[[0.9980866]]
[[0.9989784]]
[[0.9992926]]
[[0.99918455]]
[[0.99824536]]
[[0.9983993]]
[[0.99885535]]
[[0.99890065]]
[[0.9981584]]
[[0.99802774]]
[[0.99842983]]
[[0.9993274]]
[[0.99808407]]
[[0.9975643]]
[[0.9985233]]
[[0.9992537]]
[[0.9987708]]
[[0.9985191]]
[[0.9996189]]
[[0.9985833]]
[[0.99697065]]
[[0.9989905]]
[[0.99804235]]
[[0.9991239]]
[[0.99815077]]
[[0.9986319]]
[[0.9956091]]
[[0.99925476]]
[[0.99910176]]
[[0.99860686]]
[[0.9991192]]
[[0.9982552]]
[[0.997537]]
[[0.99890697]]
[[0.999196]]
[[0.9980349]]
[[0.9992943]]
[[0.99911267]]
[[0.9991775]]
[[0.998133]]
[[0.998757]]

[[0.99910176]]
[[0.9976853]]
[[0.9991673]]
[[0.99901056]]
[[0.9976374]]
[[0.99822336]]
[[0.9989644]]
[[0.99898094]]
[[0.998662]]
[[0.99905795]]
[[0.99812216]]
[[0.9990313]]
[[0.9973193]]
[[0.9991997]]
[[0.9982357]]
[[0.9993622]]
[[0.9987514]]
[[0.99901485]]
[[0.9995297]]
[[0.9989579]]
[[0.9989911]]
[[0.99890125]]
[[0.9973008]]
[[0.99850804]]
[[0.998623]]
[[0.9989341]]
[[0.9991171]]
[[0.9987571]]
[[0.99882525]]
[[0.999081]]
[[0.99909294]]
[[0.9985563]]
[[0.9975183]]
[[0.9993149]]
[[0.9976609]]
[[0.99874145]]
[[0.99870634]]
[[0.9991949]]
[[0.99956614]]
[[0.9994789]]
[[0.9991171]]
[[0.9990073]]
[[0.9989022]]
[[0.99905306]]
[[0.9959566]]
[[0.9984189]]
[[0.9983854]]
[[0.99949324]]
[[0.9974733]]
[[0.9977337]]
[[0.99724907]]
[[0.9990396]]
[[0.9988018]]
[[0.9985045]]
[[0.99946517]]
[[0.9969597]]
[[0.99897873]]
[[0.9975291]]
[[0.99898714]]
[[0.9987172]]
[[0.9988116]]
[[0.9995316]]
[[0.9983871]]
[[0.997928]]
[[0.9995197]]
[[0.9981744]]
[[0.9990941]]
[[0.99745625]]
[[0.99882895]]
[[0.998362]]
[[0

...

[[0.9986154]]
[[0.999469]]
[[0.99920875]]
[[0.99865735]]
[[0.9988494]]
[[0.9984414]]
[[0.9987233]]
[[0.99942905]]
[[0.9989078]]
[[0.9967033]]
[[0.9995333]]
[[0.99944395]]
[[0.9992022]]
[[0.99894387]]
[[0.99880064]]
[[0.9983203]]
[[0.9990102]]
[[0.9992925]]
[[0.9992254]]
[[0.9991812]]
[[0.99886584]]
[[0.999592]]
[[0.9994423]]
[[0.9980653]]
[[0.9992933]]
[[0.9983835]]
[[0.99873716]]
[[0.99862003]]
[[0.9993401]]
[[0.99885035]]
[[0.9988335]]
[[0.9978581]]
[[0.99877185]]
[[0.99900013]]
[[0.99924064]]
[[0.99806064]]
[[0.9991296]]
[[0.99878496]]
[[0.9993599]]
[[0.9991678]]
[[0.9990828]]
[[0.99833554]]
[[0.99932706]]
[[0.999393]]
[[0.9989447]]
[[0.9982368]]
[[0.9985336]]
[[0.9987172]]
[[0.9959437]]
[[0.9981729]]
[[0.9988657]]
[[0.99866056]]
[[0.9988612]]
[[0.9992162]]
[[0.9990522]]
[[0.9985045]]
[[0.99762934]]
[[0.999343]]
[[0.9992207]]
[[0.9973744]]
[[0.9986577]]
[[0.9993849]]
[[0.99976677]]
[[0.9983525]]
[[0.9973889]]
[[0.9993575]]
[[0.99918264]]
[[0.9989512]]
[[0.99796826]]
[[0.99929106]]
[

[[0.99934345]]
[[0.9986953]]
[[0.9991917]]
[[0.99811065]]
[[0.9989556]]
[[0.99851376]]
[[0.9989961]]
[[0.99836606]]
[[0.999273]]
[[0.9994179]]
[[0.9981896]]
[[0.9989172]]
[[0.9988172]]
[[0.99898463]]
[[0.999342]]
[[0.99854916]]
[[0.9990452]]
[[0.9992423]]
[[0.9984479]]
[[0.9981902]]
[[0.9991105]]
[[0.99849606]]
[[0.9983923]]
[[0.99912685]]
[[0.999288]]
[[0.9978669]]
[[0.9986539]]
[[0.998538]]
[[0.9986135]]
[[0.9993728]]
[[0.99643135]]
[[0.9980398]]
[[0.999337]]
[[0.9991322]]
[[0.99866474]]
[[0.9990777]]
[[0.9983065]]
[[0.9978254]]
[[0.9983027]]
[[0.9988587]]
[[0.9984842]]
[[0.9984471]]
[[0.999602]]
[[0.999446]]
[[0.9984861]]
[[0.9993604]]
[[0.9990821]]
[[0.99890304]]
[[0.99967134]]
[[0.99881244]]
[[0.9988225]]
[[0.99749786]]
[[0.9992415]]
[[0.99790776]]
[[0.9985019]]
[[0.9988171]]
[[0.9985808]]
[[0.99795735]]
[[0.99928015]]
[[0.99850106]]
[[0.99855655]]
[[0.9986877]]
[[0.9993119]]
[[0.99846447]]
[[0.99799573]]
[[0.9976997]]
[[0.9982765]]
[[0.99864787]]
[[0.99686146]]
[[0.99917233]]
[[0

[[0.998247]]
[[0.99727434]]
[[0.994896]]
[[0.9978461]]
[[0.99911195]]
[[0.996003]]
[[0.99904424]]
[[0.9990564]]
[[0.9992011]]
[[0.99828833]]
[[0.99914193]]
[[0.99896383]]
[[0.99924433]]
[[0.99814546]]
[[0.99882776]]
[[0.99872404]]
[[0.99941933]]
[[0.9968555]]
[[0.997502]]
[[0.9993492]]
[[0.9993197]]
[[0.9984413]]
[[0.9991611]]
[[0.9988067]]
[[0.9989612]]
[[0.99836844]]
[[0.99942577]]
[[0.9973901]]
[[0.9992617]]
[[0.99905175]]
[[0.99886143]]
[[0.999171]]
[[0.9988016]]
[[0.9988927]]
[[0.99835277]]
[[0.9978416]]
[[0.9983076]]
[[0.998546]]
[[0.9984621]]
[[0.998811]]
[[0.99915886]]
[[0.9983814]]
[[0.99870074]]
[[0.9996412]]
[[0.9977691]]
[[0.9990086]]
[[0.9988236]]
[[0.9990657]]
[[0.99937695]]
[[0.9989423]]
[[0.99924785]]
[[0.99946564]]
[[0.9984113]]
[[0.9983885]]
[[0.99909914]]
[[0.998713]]
[[0.9990663]]
[[0.9985039]]
[[0.99932957]]
[[0.9992913]]
[[0.9994068]]
[[0.99965656]]
[[0.9994791]]
[[0.99878234]]
[[0.9980223]]
[[0.99778545]]
[[0.9974952]]
[[0.99880147]]
[[0.9993667]]
[[0.9991122]]
[

[[0.99818844]]
[[0.99819666]]
[[0.9987902]]
[[0.9994197]]
[[0.9988427]]
[[0.9993882]]
[[0.9988117]]
[[0.99880004]]
[[0.9979347]]
[[0.9983329]]
[[0.9991591]]
[[0.9996282]]
[[0.99916613]]
[[0.9979163]]
[[0.997922]]
[[0.99899656]]
[[0.99858546]]
[[0.99895823]]
[[0.9979692]]
[[0.9978193]]
[[0.9989986]]
[[0.99922645]]
[[0.9990181]]
[[0.99936813]]
[[0.99708444]]
[[0.99898773]]
[[0.99924195]]
[[0.99968934]]
[[0.9974795]]
[[0.9990169]]
[[0.9989876]]
[[0.99935406]]
[[0.9980478]]
[[0.9983854]]
[[0.998987]]
[[0.9993432]]
[[0.9993492]]
[[0.99876165]]
[[0.9990871]]
[[0.9987055]]
[[0.99917716]]
[[0.9995455]]
[[0.99745876]]
[[0.9995977]]
[[0.99854136]]
[[0.9946937]]
[[0.99933285]]
[[0.99859804]]
[[0.99616563]]
[[0.997858]]
[[0.99879503]]
[[0.99911684]]
[[0.9988625]]
[[0.9993298]]
[[0.99911004]]
[[0.9987035]]
[[0.99881124]]
[[0.99914217]]
[[0.9990884]]
[[0.99778265]]
[[0.9979528]]
[[0.9993357]]
[[0.9985128]]
[[0.99953604]]
[[0.99900514]]
[[0.99866986]]
[[0.9977835]]
[[0.9991066]]
[[0.99936146]]
[[0.99

KeyboardInterrupt: 

In [31]:
print("Prediction:",sigmoid.eval(session=sess,feed_dict={X:[[18393., 168., 62., 0.],[10.0, 150., 45., 0.],[18378.,160.,65.,0.]]}))

Prediction: [[1.       ]
 [0.9952591]
 [1.       ]]


In [47]:
pr

array([[0.99937844]], dtype=float32)